In [1]:
import cv2
import mediapipe as mp
import time
from google.protobuf.json_format import MessageToDict

In [2]:
mp_pose = mp.solutions.pose

pose_detector = mp_pose.Pose(
    smooth_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [10]:
WINDOW_TITLE = "LIVE | Personal Workout Assistant"
STATUS_COLORS = {"up": (0, 255, 0), "down": (0, 0, 255)}
KPT_COLOR = (255, 255, 255)
TEXT_COLOR = (0, 0, 0)

In [31]:
source = cv2.VideoCapture('sources/Workout.mp4')
count = 0
status = "N/A"
lifting_indicate_box_fill = -1
pre_frame_ret_time, new_frame_ret_time, fps = 0, -1, -1

cv2.namedWindow(WINDOW_TITLE)
cv2.moveWindow(WINDOW_TITLE, 350, 17)

while True:
    ret, frame = source.read()
    
    if ret:
        new_frame_ret_time = time.time()
        fps = int(1 / (new_frame_ret_time - pre_frame_ret_time))
        pre_frame_ret_time = new_frame_ret_time

        image = cv2.resize(frame, (650, 650))
        hand_landmark_cordinates = get_hand_landmark_cordinates(image)
        landmarks = hand_landmark_cordinates["left"]

        if len(landmarks) == 3:
            _15_landmark = landmarks[2]
            _13_landmark = landmarks[1]
            _11_landmark = landmarks[0]
            
            if _15_landmark[1] < _13_landmark[1]:
                if status != "UP":
                    count += 1
                    status = "UP"
            else:
                status = "DOWN"
                
            status_color = STATUS_COLORS["up"]
            if status == "DOWN":
                status_color = STATUS_COLORS["down"]

            cv2.putText(image, "UP", (90, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), 2)
            cv2.putText(image, "DOWN", (90, 600), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), 2)
            cv2.rectangle(image, (40, 200), (80, 600), status_color, 2, 1)
            
            lifting_indicate_box_fill = int((_15_landmark[1] / (_13_landmark[1] - 50)) * 400) - 200
            
            if status == "UP":
                lifting_indicate_box_fill //= 2
                if 400 - lifting_indicate_box_fill >= 200:
                    image[600:400:-1, 42:79] = (0, 255, 0)
                    image[400:400-lifting_indicate_box_fill:-1, 42:79] = (0, 255, 0)
            elif status == "DOWN":
                lifting_indicate_box_fill += 400
                lifting_indicate_box_fill = abs(lifting_indicate_box_fill) // 2
                if 600 - lifting_indicate_box_fill >= 200:
                    image[600:600-lifting_indicate_box_fill:-1, 42:79] = (0, 0, 255)

            cv2.putText(image, "({0}, {1})".format(_13_landmark[2], _13_landmark[3]), (_13_landmark[2] + 12, _13_landmark[3]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, TEXT_COLOR, 2)
            cv2.putText(image, "({0}, {1})".format(_15_landmark[2], _15_landmark[3]), (_15_landmark[2] + 12, _15_landmark[3]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, TEXT_COLOR, 2)
            cv2.putText(image, "({0}, {1})".format(_11_landmark[2], _11_landmark[3]), (_11_landmark[2] + 12, _11_landmark[3]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, TEXT_COLOR, 2)
            
            cv2.line(image, (_11_landmark[2], _11_landmark[3]), (_13_landmark[2], _13_landmark[3]), status_color, 2)
            cv2.line(image, (_13_landmark[2], _13_landmark[3]), (_15_landmark[2], _15_landmark[3]), status_color, 2)
            
            cv2.circle(image, (_13_landmark[2], _13_landmark[3]), 3, KPT_COLOR, -1)
            cv2.circle(image, (_13_landmark[2], _13_landmark[3]), 6, KPT_COLOR, 1)
            cv2.circle(image, (_15_landmark[2], _15_landmark[3]), 3, KPT_COLOR, -1)
            cv2.circle(image, (_15_landmark[2], _15_landmark[3]), 6, KPT_COLOR, 1)
            cv2.circle(image, (_11_landmark[2], _11_landmark[3]), 3, KPT_COLOR, -1)
            cv2.circle(image, (_11_landmark[2], _11_landmark[3]), 6, KPT_COLOR, 1)

        image[:140, 0:270] = (255, 255, 255)

        cv2.putText(image, "Status: ", (40, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
        cv2.putText(image, "Count: ", (40, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 3)
        cv2.putText(image, "FPS:{0}".format(fps), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        cv2.putText(image, status, (155, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2)
        cv2.putText(image, str(count), (155, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                
        cv2.imshow(WINDOW_TITLE, image)
        k = cv2.waitKey(45)
        if k == 27:
            break
    else:
        break

source.release()
cv2.destroyAllWindows()

In [24]:
def get_hand_landmark_cordinates(image):
        return_cordinate_ids = [11, 13, 15, 12, 14, 16]
        left_right_hand_landmarks = {"left": [], "right": []}

        height, width, _ = image.shape
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = pose_detector.process(image)
        image.flags.writeable = True

        if results.pose_landmarks:
                pose_landmarks = MessageToDict(results.pose_landmarks)["landmark"]
                for i, landmark in enumerate(MessageToDict(results.pose_world_landmarks)["landmark"]):
                        img_x = int(pose_landmarks[i]["x"] * width)
                        img_y = int(pose_landmarks[i]["y"] * height)
                        x = int(landmark["x"] * width)
                        y = int(landmark["y"] * height)
                        visibility = landmark["visibility"] * 100
                        
                        if i in return_cordinate_ids[:3]:
                                if visibility < 80.0:
                                        left_right_hand_landmarks["left"] = []
                                else:
                                        left_right_hand_landmarks["left"].append((x, y, img_x, img_y, visibility))
        return left_right_hand_landmarks

                                